## محمد مهدی حاجی عباسی
در این بخش قصد داریم یک مشکل رو بررسی کنیم و آن وقتی که سایز گراف هایی که داریم بزرگ باشه و در این حالت اجرا گرفتن از آن دشوار خواهد بود چرا که ابزار هایی که داریم محدود هستند.
مثالی که اینجا استفاده شده گراف کوچیکی هست و فقط برای اموزش استفاده شده طبعا این روش روی راس ها و یال های میلیونی مورد استفاده قرار میگیرد.

مبنای این روش مقاله :

**Cluster-GCN** ([Chiang et al. (2019)](https://arxiv.org/abs/1905.07953)

هست

In [1]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.0.1+cu118
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

dataset = Planetoid(root='data/Planetoid', name='PubMed', transform=NormalizeFeatures())

print()
print(f'Dataset: {dataset}:')
print('==================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===============================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.3f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Processing...



Dataset: PubMed():
Number of graphs: 1
Number of features: 500
Number of classes: 3

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])
Number of nodes: 19717
Number of edges: 88648
Average node degree: 4.50
Number of training nodes: 60
Training node label rate: 0.003
Has isolated nodes: False
Has self-loops: False
Is undirected: True


Done!


In [ ]:
from torch_geometric.loader import ClusterData, ClusterLoader
# با استفاده از این کتابخانه دیتا ها به 128 زیر گراف تبدیل میشوند

torch.manual_seed(12345)
cluster_data = ClusterData(data, num_parts=128)  # 1. Create subgraphs.
#ودر هر مرحله 32 تا از گراف ها آموزش می بیند
train_loader = ClusterLoader(cluster_data, batch_size=32, shuffle=True)  # 2. Stochastic partioning scheme.

print()
total_num_nodes = 0
for step, sub_data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of nodes in the current batch: {sub_data.num_nodes}')
    print(sub_data)
    print()
    total_num_nodes += sub_data.num_nodes

print(f'Iterated over {total_num_nodes} of {data.num_nodes} nodes!')

Computing METIS partitioning...



Step 1:
Number of nodes in the current batch: 4928
Data(x=[4928, 500], y=[4928], train_mask=[4928], val_mask=[4928], test_mask=[4928], edge_index=[2, 16174])

Step 2:
Number of nodes in the current batch: 4937
Data(x=[4937, 500], y=[4937], train_mask=[4937], val_mask=[4937], test_mask=[4937], edge_index=[2, 17832])

Step 3:
Number of nodes in the current batch: 4927
Data(x=[4927, 500], y=[4927], train_mask=[4927], val_mask=[4927], test_mask=[4927], edge_index=[2, 14712])

Step 4:
Number of nodes in the current batch: 4925
Data(x=[4925, 500], y=[4925], train_mask=[4925], val_mask=[4925], test_mask=[4925], edge_index=[2, 18006])

Iterated over 19717 of 19717 nodes!


Done!


در کنار این کار که انجام دادیم محاسبات رو کاهش داد کار دیگه ای که انجام میشه.
استفاده ازdroputهست
این کار از برزو رسانی یکسری از وزن ها جلو گیری می کند.

که  این کار نیز هزینه محاسبات رو کاهش میدهد.

In [ ]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
#ساخت شبکه
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, dataset.num_classes)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GCN(hidden_channels=16)
print(model)

GCN(
  (conv1): GCNConv(500, 16)
  (conv2): GCNConv(16, 3)
)


آموزش این شبکه عصبی گرافی به شکل قابل توجهی شبیه به آموزش شبکه‌های عصبی گرافی برای کلاس‌بندی گراف است. به جای عملکرد بر روی گراف به صورت کامل، ما در حال حاضر بر روی هر مینی‌بچ تکرار می‌کنیم و هر بچه را به طور مستقل بهینه‌سازی می‌کنیم:



In [ ]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()

      for sub_data in train_loader:  # Iterate over each mini-batch.
          out = model(sub_data.x, sub_data.edge_index)  # Perform a single forward pass.
          loss = criterion(out[sub_data.train_mask], sub_data.y[sub_data.train_mask])  # Compute the loss solely based on the training nodes.
          loss.backward()  # Derive gradients.
          optimizer.step()  # Update parameters based on gradients.
          optimizer.zero_grad()  # Clear gradients.

def test():
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      
      accs = []
      for mask in [data.train_mask, data.val_mask, data.test_mask]:
          correct = pred[mask] == data.y[mask]  # Check against ground-truth labels.
          accs.append(int(correct.sum()) / int(mask.sum()))  # Derive ratio of correct predictions.
      return accs

for epoch in range(1, 51):
    loss = train()
    train_acc, val_acc, test_acc = test()
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train: 0.3333, Val Acc: 0.4160, Test Acc: 0.4070
Epoch: 002, Train: 0.4833, Val Acc: 0.4420, Test Acc: 0.4270
Epoch: 003, Train: 0.7333, Val Acc: 0.5220, Test Acc: 0.5160
Epoch: 004, Train: 0.8333, Val Acc: 0.5920, Test Acc: 0.5930
Epoch: 005, Train: 0.9167, Val Acc: 0.6920, Test Acc: 0.6810
Epoch: 006, Train: 0.9000, Val Acc: 0.6680, Test Acc: 0.6680
Epoch: 007, Train: 0.8833, Val Acc: 0.6520, Test Acc: 0.6530
Epoch: 008, Train: 0.9167, Val Acc: 0.7020, Test Acc: 0.6940
Epoch: 009, Train: 0.9333, Val Acc: 0.7340, Test Acc: 0.7290
Epoch: 010, Train: 0.9333, Val Acc: 0.6880, Test Acc: 0.6900
Epoch: 011, Train: 0.9500, Val Acc: 0.7240, Test Acc: 0.7330
Epoch: 012, Train: 0.9500, Val Acc: 0.7500, Test Acc: 0.7300
Epoch: 013, Train: 0.9667, Val Acc: 0.7680, Test Acc: 0.7530
Epoch: 014, Train: 0.9667, Val Acc: 0.7600, Test Acc: 0.7450
Epoch: 015, Train: 0.9667, Val Acc: 0.7680, Test Acc: 0.7550
Epoch: 016, Train: 0.9667, Val Acc: 0.7660, Test Acc: 0.7630
Epoch: 017, Train: 0.966

و در نهایت لینک چند منبع مناسب برای مطالعه بیشتر کد های مربوط به این زمینه

**[PyG's documentation](https://pytorch-geometric.readthedocs.io/en/latest/?badge=latest)**


**[مثال های بیشتر](https://github.com/rusty1s/pytorch_geometric/tree/master/examples)**